In [61]:
def type_of_rating(data):
    facilities = []
    for row in data[1:]:
        facility1 = row[0]
        facility = str(facility1).lower()
        print(facility)
        if 'short term' in facility or 'long term / short term' in facility:
            facilities.append('ST')
        elif 'long term' in facility or 'long-term'in facility or 'long - term'in facility:
            facilities.append('LT')
        elif 'unallocated term' in facility:
            facilities.append('UT')
        else:
            facilities.append('ST')
    return facilities

import re
import numpy as np

def extract_ratings_from_data(data):
    rating_pattern = re.compile(r'\b(?:[A-Z]{1,3}[+-]?/[A-Z0-9+-]+(?:/[A-Z0-9+-]+)?(?:;Negative)?)\b|[A-Z0-9+-]+(?:;Negative)?')
    ratings = []
    for row in data:
        row_ratings = []
        for cell in row:
            if isinstance(cell, str): 
                matches = rating_pattern.findall(cell)
                valid_matches = [match for match in matches if len(match) > 1 and not match.isdigit() and match not in ['CARE', 'ICRA', 'Commercial paper (CP)','CP','CE']]
                row_ratings.extend(valid_matches)
        if row_ratings:
            ratings.append('/'.join(row_ratings))
    
    return ratings

# def extract_type_of_facilities_care(data):
#     facilities = []
#     pattern = re.compile(r'Commercial paper \(CP\)|^[A-Za-z\s\-\/\(\)]+(?:Bank Facilities|bank facilities|Term Loan|term loan|Commercial paper|commercial paper|Amount)[@#]?$')
#     for row in data:
#         if isinstance(row[0], str) and pattern.match(row[0]):
#             facilities.append(row[0])
    
#     return facilities

def extract_type_of_facilities(data):
    facilities = []
    pattern = re.compile(r'Commercial paper \(CP\)|^[A-Za-z\s\-\/\(\)]+(?:Bank Facilities|Term Loan|Commercial paper|Amount)$')
    for row in data:
        if isinstance(row[0], str) and pattern.match(row[0]):
            facilities.append(row[0])
    
    return facilities

In [62]:

def company_name(companies):
    com_name = []
    if 'CARE' in companies[0] or 'ICRA' in companies[0]:
        companies.pop(0)
    new_first_company = companies[0]
    new_first_company = re.sub(r'Press Release\s+', '', new_first_company, flags=re.IGNORECASE)
    new_first_company = re.sub(r'^\d+\s+', '', new_first_company)
    pattern = re.compile(r'\b[A-Za-z\s]+(?:Limited|Ltd|Corporation|Corp|Incorporated|Inc|LLC|LLP)\b', re.IGNORECASE)
    matches = pattern.findall(new_first_company)
    expected_company_name = matches[0] if matches else ""
    expected_company_name = expected_company_name.replace('\n', ' ')
    com_name.append(expected_company_name)
    return com_name


def outlook(data):
    ratings = []
    for row in data[1:]:
        rating = row[2]
        if 'Stable' in rating:
            ratings.append('Stable')
        elif 'Positive' in rating or 'Negative' in rating:
            ratings.append(rating.split(';')[1].strip())
        else:
            ratings.append('-')

    return ratings


from PyPDF2 import PdfReader
import numpy as np

def remove_commas(data):
    for row in data:
        for i in range(len(row)):
            if isinstance(row[i], str) and row[i].replace(',', '').replace('.', '').isdigit():
                row[i] = row[i].replace(',', '')
    return data

from PyPDF2 import PdfReader
def extract_text_from_pdf(pdf_file_path):
    reader = PdfReader(pdf_file_path)
    page = reader.pages[0]
    text = page.extract_text()
    return text

def find_agency_names(pdf_file_path):
    text = extract_text_from_pdf(pdf_file_path)
    agencies = []
    if 'CARE' in text:
        agencies.append('CARE')
    if 'ICRA' in text:
        agencies.append('ICRA')
    if 'Brickwork rating' in text or 'brickworkrating' in text:
        agencies.append('Brickworks')
    return agencies

In [63]:
# ## code for contact details

import re
from PyPDF2 import PdfReader

def extract_text_from_last_pages(pdf_file_path, num_pages=3):
    reader = PdfReader(pdf_file_path)
    text = ""
    total_pages = len(reader.pages)
    for page_num in range(total_pages - num_pages, total_pages):
        text += reader.pages[page_num].extract_text()
    return text

def extract_analyst_contacts(pdf_file_path):
    text = extract_text_from_last_pages(pdf_file_path)
    
    if "ANALYST CONTACT" in text:
        name_pattern = re.compile(r'(?<=ANALYST CONTACTS\s)[A-Za-z\s]+')
        phone_pattern = re.compile(r'\+\d{2}\s\d{2}\s\d{4}\s\d{4}')
        email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
        
        names = name_pattern.findall(text)
        phones = phone_pattern.findall(text)
        emails = email_pattern.findall(text)
        
        names_list = []
        emails_list = []
        contact_numbers_list = []
        
        for name in names:
            name = name.strip()
            phone_match = phone_pattern.search(text, text.find(name))
            phone = phone_match.group(0) if phone_match else None
            email_match = email_pattern.search(text, text.find(name))
            email = email_match.group(0) if email_match else None
            
            names_list.append(name)
            contact_numbers_list.append(phone)
            emails_list.append(email)

        return names_list, emails_list, contact_numbers_list
    else:
        return ['-'], ['-'], ['-']


In [64]:
def extract_all_amount(data):
    short_term_amounts = []
    long_term_amounts = []
    unallocated_amounts = []
    
    for row in data:
        if isinstance(row[1], str) and row[1].replace('.', '', 1).isdigit():
            amount = float(row[1])
            facility1 = row[0]
            facility = facility1.lower()
            
            if 'short term' in facility or 'long term / short term' in facility or 'short-term' in facility:
                short_term_amounts.append(amount)
                long_term_amounts.append(0.00)
                unallocated_amounts.append(0.00)
            elif 'long term' in facility or 'long-term' in facility:
                long_term_amounts.append(amount)
                short_term_amounts.append(0.00)
                unallocated_amounts.append(0.00)

            elif 'lc/bg' in facility or 'LC/BG' in facility:
                short_term_amounts.append(amount)
                long_term_amounts.append(0.00)
                unallocated_amounts.append(0.00)

            elif 'term loan' in facility or 'Term loan' in facility:
                long_term_amounts.append(amount)
                short_term_amounts.append(0.00)
                unallocated_amounts.append(0.00)
            
            elif 'unallocated term' in facility or 'propose' in facility:
                unallocated_amounts.append(amount)
                short_term_amounts.append(0.00)
                long_term_amounts.append(0.00)

            else:
                short_term_amounts.append(amount)
                unallocated_amounts.append(0.00)
                long_term_amounts.append(0.00)
    
    return short_term_amounts, long_term_amounts, unallocated_amounts


In [65]:

def remove_hyphens(data):
    for row in data:
        row[0] = str(row[0]).replace('–', '').replace('-', '')
    return data

In [67]:
import pandas as pd
# xl_path = r"D:\Python\ajit\project_credit_rating\try\auto\excel\Adani Agri Fresh Limited 1.xlsx"
# pdf_file_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Agri Fresh Limited 1.pdf"

# pdf_file_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Transmission Limited.pdf"
# xl_path = r'D:\Python\ajit\project_credit_rating\try\auto\excel\Adani Transmission Limited.xlsx'

pdf_file_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Hazira Port Limited.pdf"
xl_path = r"D:\Python\ajit\project_credit_rating\try\excel\Adani Hazira Port Limited.xlsx"

com1 = []
age=[]
name = []
email = []
contact_number = []
ty_faci = []
xls = pd.ExcelFile(xl_path)
table_name = xls.sheet_names
df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
a1 = df1.values.tolist()
filtered_rs = [row for row in a1 if '(Rs. crore)' not in row]
filtered_total = [row for row in filtered_rs if 'Total' not in row]

data = remove_hyphens(filtered_total)

rating_type = type_of_rating(data)
data = remove_commas(data)
outlook1 = outlook(data)
short_term_amounts, long_term_amounts, unallocated_amounts = extract_all_amount(data)
ratings = extract_ratings_from_data(data)
rat1 = []
if ratings != rating_type:
    rat1.extend(ratings * len(rating_type))
else:
    rat1 = ratings

type_facility=extract_type_of_facilities(data)
ty_faci.extend(type_facility*len(rating_type))
print(type_facility)
names, emails, contact_numbers = extract_analyst_contacts(pdf_file_path)
name.extend(names*len(rating_type))
email.extend(emails*len(rating_type))
contact_number.extend(contact_numbers*len(rating_type))
agencies = find_agency_names(pdf_file_path)
text = extract_text_from_pdf(pdf_file_path)
pattern = re.compile(r'\b[A-Za-z0-9\s]+(?:Limited|Ltd|Corporation|Corp|Incorporated|Inc|LLC|LLP)\b', re.IGNORECASE)
companies = pattern.findall(text)
com = company_name(companies)
com1.extend(com * len(rating_type))
age.extend(agencies * len(rating_type))

print(len(com1),len(age),len(ty_faci),len(rating_type),len(outlook1),len(rat1),len(short_term_amounts),
      len(long_term_amounts),len(unallocated_amounts),len(name),len(email),len(contact_number))

data = {
        'Name of Adani Entity': com1,
        'Name of the Rating Agency': age,
        'Types of Rating': rating_type,
        'Ratings': rat1,
        'Outlook': outlook1,
        'Type of Facility': ty_faci,
        'Rated Outstanding Long Term Amount': long_term_amounts,
        'Rated Outstanding Short Term Amount': short_term_amounts,
        'Rated Outstanding Unallocated Amount': unallocated_amounts,
        'Credit Rating Analyst Name': name,
        'Credit Rating Analyst Email-Id': email,
        'Credit Rating Analyst Contact': contact_number
        }

# Create the DataFrame
df = pd.DataFrame(data)
df

fund based  term loan
['Fund Based  Term Loan']
1 1 1 1 1 1 1 1 1 1 1 1


,Name of Adani Entity,Name of the Rating Agency,Types of Rating,Ratings,Outlook,Type of Facility,Rated Outstanding Long Term Amount,Rated Outstanding Short Term Amount,Rated Outstanding Unallocated Amount,Credit Rating Analyst Name,Credit Rating Analyst Email-Id,Credit Rating Analyst Contact
0,Adani Hazira Port Limited,ICRA,ST,AA+,-,Fund Based Term Loan,600.0,0.0,0.0,Sabyasachi Majumdar,sabyasachi@icraindia.com,+91 12 4454 5304


In [9]:
((com1),(age),(ty_faci),(rating_type),(outlook1),(rat1),(short_term_amounts),
      (long_term_amounts),(unallocated_amounts),(name),(email),(contact_number))

(['  Adani Transmission  Limited'],
 ['ICRA'],
 ['Commercial paper (CP)'],
 ['ST'],
 ['-'],
 ['A1+'],
 [1000.0],
 [0.0],
 [0.0],
 ['Sabyasachi Majumdar'],
 ['girishkumar@icraindia.com'],
 ['+91 22 6114 3441'])

In [36]:
import os
import pandas as pd
import re

pdf_directory = r"D:\Python\ajit\project_credit_rating\try\auto\icra\pdf"
excel_directory = r"D:\Python\ajit\project_credit_rating\try\auto\icra\excel"
op_csv = r'D:\Python\ajit\project_credit_rating\try\auto\icra\op_csv'

for excel_file in os.listdir(excel_directory):
    if excel_file.endswith(".xlsx"):
        xl_path = os.path.join(excel_directory, excel_file)
        pdf_file_path = os.path.join(pdf_directory, excel_file.replace(".xlsx", ".pdf"))      
        com1 = []
        age=[]
        name = []
        email = []
        contact_number = []
        ty_faci = []
        xls = pd.ExcelFile(xl_path)
        table_name = xls.sheet_names
        df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
        a1 = df1.values.tolist()
        filtered_rs = [row for row in a1 if '(Rs. crore)' not in row]
        filtered_total = [row for row in filtered_rs if 'Total' not in row]

        data = remove_hyphens(filtered_total)

        rating_type = type_of_rating(data)
        data = remove_commas(data)
        outlook1 = outlook(data)
        short_term_amounts, long_term_amounts, unallocated_amounts = extract_all_amount(data)
        ratings = extract_ratings_from_data(data)
        rat1 = []
        if ratings != rating_type:
            rat1.extend(ratings * len(rating_type))
        else:
            rat1 = ratings

        type_facility=extract_type_of_facilities(data)
        ty_faci.extend(type_facility*len(rating_type))
        print(type_facility)
        names, emails, contact_numbers = extract_analyst_contacts(pdf_file_path)
        name.extend(names*len(rating_type))
        email.extend(emails*len(rating_type))
        contact_number.extend(contact_numbers*len(rating_type))
        agencies = find_agency_names(pdf_file_path)
        text = extract_text_from_pdf(pdf_file_path)
        pattern = re.compile(r'\b[A-Za-z0-9\s]+(?:Limited|Ltd|Corporation|Corp|Incorporated|Inc|LLC|LLP)\b', re.IGNORECASE)
        companies = pattern.findall(text)
        com = company_name(companies)
        com1.extend(com * len(rating_type))
        age.extend(agencies * len(rating_type))

        print(len(com1),len(age),len(ty_faci),len(rating_type),len(outlook1),len(rat1),len(short_term_amounts),
            len(long_term_amounts),len(unallocated_amounts),len(name),len(email),len(contact_number))

        data = {
                'Name of Adani Entity': com1,
                'Name of the Rating Agency': age,
                'Types of Rating': rating_type,
                'Ratings': rat1,
                'Outlook': outlook1,
                'Type of Facility': ty_faci,
                'Rated Outstanding Long Term Amount': long_term_amounts,
                'Rated Outstanding Short Term Amount': short_term_amounts,
                'Rated Outstanding Unallocated Amount': unallocated_amounts,
                'Credit Rating Analyst Name': name,
                'Credit Rating Analyst Email-Id': email,
                'Credit Rating Analyst Contact': contact_number
                }


        df = pd.DataFrame(data)
        print(df)
        print('\n=====================')

        output_csv_path = os.path.join(op_csv, excel_file.replace(".xlsx", ".csv"))
        df.to_csv(output_csv_path, index=False)

print("Data has been successfully saved to separate CSV files for each PDF.")


fund based  term loan
['Fund Based  Term Loan']
1 1 1 1 1 1 1 1 1 1 1 1
          Name of Adani Entity Name of the Rating Agency Types of Rating  \
0    Adani Hazira Port Limited                      ICRA              ST   

  Ratings Outlook       Type of Facility  Rated Outstanding Long Term Amount  \
0     AA+       -  Fund Based  Term Loan                               600.0   

   Rated Outstanding Short Term Amount  Rated Outstanding Unallocated Amount  \
0                                  0.0                                   0.0   

  Credit Rating Analyst Name Credit Rating Analyst Email-Id  \
0        Sabyasachi Majumdar       sabyasachi@icraindia.com   

  Credit Rating Analyst Contact  
0              +91 12 4454 5304  

commercial paper (cp)
['Commercial paper (CP)']
1 1 1 1 1 1 1 1 1 1 1 1
            Name of Adani Entity Name of the Rating Agency Types of Rating  \
0    Adani Transmission  Limited                      ICRA              ST   

  Ratings Outlook       Type

In [28]:
# ((com1),(age),(ty_faci),(rating_type),(outlook1),(rat1),(short_term_amounts),
#       (long_term_amounts),(unallocated_amounts),(name),(email),(contact_number))

In [52]:
# ## code for contact details

# # pdf_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Agri Fresh Limited 1.pdf"
# pdf_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Transmission Limited.pdf"
# # pdf_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Hazira Port Limited.pdf"

# import re
# from PyPDF2 import PdfReader

# def extract_text_from_last_pages(pdf_file_path, num_pages=3):
#     reader = PdfReader(pdf_file_path)
#     text = ""
#     total_pages = len(reader.pages)
#     for page_num in range(total_pages - num_pages, total_pages):
#         text += reader.pages[page_num].extract_text()
#     return text

# def extract_analyst_contacts(pdf_file_path):
#     text = extract_text_from_last_pages(pdf_file_path)
    
#     if "ANALYST CONTACT" in text:
#         # name_pattern = re.compile(r'(?<=\n)[A-Za-z\s]+(?=\n\+\d{2}\s\d{2,3}\s\d{7,10})')
#         # email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
#         # phone_pattern = re.compile(r'\+\d{2}\s\d{2,3}\s\d{7,10}')

#         name_pattern = re.compile(r'(?<=ANALYST CONTACTS\s)[A-Za-z\s]+')
#         phone_pattern = re.compile(r'\+\d{2}\s\d{2}\s\d{4}\s\d{4}')
#         email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
        
#         names = name_pattern.findall(text)
#         emails = email_pattern.findall(text)
#         phones = phone_pattern.findall(text)
#         names_list = []
#         emails_list = []
#         contact_numbers_list = []
        
#         for name, phone in zip(names, phones):
#             email_match = email_pattern.search(text, text.find(phone))
#             email = email_match.group(0) if email_match else None
#             names_list.append(name.strip())
#             emails_list.append(email)
#             contact_numbers_list.append(phone)

#         return names_list, emails_list, contact_numbers_list
#     else:
#         return ['-'], ['-'], ['-']

# names, emails, contact_numbers = extract_analyst_contacts(pdf_path)
# print("Names:", names)
# print("Emails:", emails)
# print("Contact Numbers:", contact_numbers)


In [60]:
# import re
# from PyPDF2 import PdfReader

# def extract_text_from_last_pages(pdf_file_path, num_pages=3):
#     reader = PdfReader(pdf_file_path)
#     text = ""
#     total_pages = len(reader.pages)
#     for page_num in range(total_pages - num_pages, total_pages):
#         text += reader.pages[page_num].extract_text()
#     return text

# def extract_analyst_contacts(pdf_file_path):
#     text = extract_text_from_last_pages(pdf_file_path)
    
#     if "ANALYST CONTACT" in text:
#         name_pattern = re.compile(r'(?<=ANALYST CONTACTS\s)[A-Za-z\s]+')
#         phone_pattern = re.compile(r'\+\d{2}\s\d{2}\s\d{4}\s\d{4}')
#         email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
        
#         names = name_pattern.findall(text)
#         phones = phone_pattern.findall(text)
#         emails = email_pattern.findall(text)
        
#         names_list = []
#         emails_list = []
#         contact_numbers_list = []
        
#         for name in names:
#             name = name.strip()
#             phone_match = phone_pattern.search(text, text.find(name))
#             phone = phone_match.group(0) if phone_match else None
#             email_match = email_pattern.search(text, text.find(name))
#             email = email_match.group(0) if email_match else None
            
#             names_list.append(name)
#             contact_numbers_list.append(phone)
#             emails_list.append(email)

#         return names_list, emails_list, contact_numbers_list
#     else:
#         return ['-'], ['-'], ['-']

# pdf_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Transmission Limited.pdf"
# # pdf_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Hazira Port Limited.pdf"
# names, emails, contact_numbers = extract_analyst_contacts(pdf_path)
# print("Names:", names)
# print("Emails:", emails)
# print("Contact Numbers:", contact_numbers)
